In [5]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

Reimplement your model from the previous lesson.

In [6]:
house_prices_df = pd.read_csv("https://djl-lms-assets.s3.eu-central-1.amazonaws.com/datasets/house_prices.csv", sep = ";")
house_prices_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
miss_val_ind = (house_prices_df.isnull().sum().sort_values(ascending=False)/len(house_prices_df)*100).head(10).index
house_prices_df.drop(miss_val_ind[0:5],axis=1,inplace=True)
house_prices_df["LotFrontage"].fillna(house_prices_df["LotFrontage"].mean(),inplace=True)
house_prices_df.dropna(inplace=True)
house_prices_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338 entries, 0 to 1459
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1338 non-null   int64  
 1   MSSubClass     1338 non-null   int64  
 2   MSZoning       1338 non-null   object 
 3   LotFrontage    1338 non-null   float64
 4   LotArea        1338 non-null   int64  
 5   Street         1338 non-null   object 
 6   LotShape       1338 non-null   object 
 7   LandContour    1338 non-null   object 
 8   Utilities      1338 non-null   object 
 9   LotConfig      1338 non-null   object 
 10  LandSlope      1338 non-null   object 
 11  Neighborhood   1338 non-null   object 
 12  Condition1     1338 non-null   object 
 13  Condition2     1338 non-null   object 
 14  BldgType       1338 non-null   object 
 15  HouseStyle     1338 non-null   object 
 16  OverallQual    1338 non-null   int64  
 17  OverallCond    1338 non-null   int64  
 18  YearBuil

In [9]:
obj_ind = house_prices_df.dtypes[house_prices_df.dtypes=="O"].index
for i in obj_ind:
    if house_prices_df[i].nunique()>5:
        house_prices_df.drop(i,axis=1,inplace=True)

In [10]:
obj_ind = house_prices_df.dtypes[house_prices_df.dtypes=="O"]
corr_ind = house_prices_df.corr()["SalePrice"].sort_values(ascending=False).index
house_prices_df.drop(corr_ind[11:],axis=1,inplace=True)
complet_df = pd.get_dummies(house_prices_df,drop_first=True)

In [11]:
X = complet_df.drop("SalePrice",axis=1)
y = complet_df["SalePrice"]
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42)

Try OLS, Lasso, Ridge and ElasticNet regressions using the same model specification. This time, you need to do k-fold cross-validation to choose the best hyperparameter values for your models. Which model is the best? Why?

In [12]:
X_cons = sm.add_constant(X_train)
results = sm.OLS(y_train, X_cons).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.906
Model:                            OLS   Adj. R-squared:                  0.898
Method:                 Least Squares   F-statistic:                     109.4
Date:                Thu, 17 Feb 2022   Prob (F-statistic):               0.00
Time:                        15:49:19   Log-Likelihood:                -10829.
No. Observations:                 936   AIC:                         2.181e+04
Df Residuals:                     859   BIC:                         2.219e+04
Df Model:                          76                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const              -6.223e+05   1.59e+05     -3.925      0.000   -9.34e+05   -3.11e+05
OverallQual         8364.8186   1310.551      6.383      0.000    5792.562    1.09e+04
YearBuilt            176.3887     81.121      2.174      0.030      17.169     335.608
YearRemodAdd         259.8808     71.884      3.615      0.000     118.791     400.970
TotalBsmtSF           56.2628      6.198      9.078      0.000      44.098      68.427
1stFlrSF             -25.0535      6.315     -3.968      0.000     -37.447     -12.660
GrLivArea             82.8421      4.191     19.767      0.000      74.617      91.068
FullBath           -5499.8410   2599.133     -2.116      0.035   -1.06e+04    -398.446
TotRmsAbvGrd       -3273.2095   1149.882     -2.847      0.005   -5530.116   -1016.303
GarageCars          1784.8934   2999.860      0.595      0.552   -4103.019    7672.806
GarageArea            25.8092      9.968      2.589      0.010       6.245      45.373
MSZoning_FV         2.342e+04    1.7e+04      1.377      0.169   -9957.903    5.68e+04
MSZoning_RH         1.361e+04   1.94e+04      0.703      0.482   -2.44e+04    5.16e+04
MSZoning_RL         1.755e+04   1.63e+04      1.075      0.283   -1.45e+04    4.96e+04
MSZoning_RM         9668.7434   1.64e+04      0.588      0.557   -2.26e+04     4.2e+04
Street_Pave         1.141e+04   3.23e+04      0.353      0.724    -5.2e+04    7.48e+04
LotShape_IR2        7655.0169   5361.008      1.428      0.154   -2867.192    1.82e+04
LotShape_IR3         650.8620   1.14e+04      0.057      0.954   -2.17e+04     2.3e+04
LotShape_Reg       -1287.0996   2155.172     -0.597      0.551   -5517.118    2942.919
LandContour_HLS     4082.8692   7164.268      0.570      0.569   -9978.652    1.81e+04
LandContour_Low      -11.4627   8407.909     -0.001      0.999   -1.65e+04    1.65e+04
LandContour_Lvl      804.2517   5332.872      0.151      0.880   -9662.734    1.13e+04
Utilities_NoSeWa   -3.054e-10   2.17e-10     -1.408      0.159   -7.31e-10     1.2e-10
LotConfig_CulDSac   6386.3190   4182.985      1.527      0.127   -1823.749    1.46e+04
LotConfig_FR2       -1.11e+04   5579.143     -1.989      0.047    -2.2e+04    -145.427
LotConfig_FR3      -3680.9825   1.41e+04     -0.261      0.794   -3.13e+04     2.4e+04
LotConfig_Inside   -1493.0852   2421.736     -0.617      0.538   -6246.298    3260.127
LandSlope_Mod       5031.2784   5461.481      0.921      0.357   -5688.132    1.58e+04
LandSlope_Sev        353.2592   1.25e+04      0.028      0.978   -2.42e+04     2.5e+04
BldgType_2fmCon    -7375.3500   9319.684     -0.791      0.429   -2.57e+04    1.09e+04
BldgType_Duplex    -5.063e+04   7242.448     -6.991      0.000   -6.48e+04   -3.64e+04
BldgType_Twnhs     -1.495e+04   5818.061     -2.570      0.010   -2.64e+04   -3532.614
BldgType_TwnhsE    -1.097e+04   3890.5

In [13]:
from sklearn.linear_model import LassoCV

In [30]:
reg = LassoCV(cv=5, random_state=0).fit(X_train, y_train)
print(f"Train Score:{reg.score(X_train, y_train)}")
print(f"Test Score:{reg.score(X_test, y_test)}")
print(f"Best Alpha value:{reg.alpha_}")

Train Score:0.8058888157800729
Test Score:0.1880599137252532
Best Alpha value:32955.7779249055


In [31]:
from sklearn.linear_model import RidgeCV

In [47]:
ridcv =RidgeCV(cv=5).fit(X_train, y_train)
print(f"Train Score:{ridcv.score(X_train, y_train)}")
print(f"Test Score:{ridcv.score(X_test, y_test)}")
print(f"Best Alpha value:{ridcv.alpha_}")
ridcv.

Train Score:0.9054965542740454
Test Score:0.33494370810754726
Best Alpha value:1.0


In [48]:
from sklearn.linear_model import ElasticNetCV

In [49]:
elascv = ElasticNetCV(cv=5).fit(X_train, y_train)
print(f"Train Score:{elascv.score(X_train, y_train)}")
print(f"Test Score:{elascv.score(X_test, y_test)}")
print(f"Best Alpha value:{elascv.alpha_}")
print(f"Best l1_ratio value:{elascv.l1_ratio}")

Train Score:0.743279573133629
Test Score:0.16618995134774261
Best Alpha value:65911.55584981102
Best l1_ratio value:0.5


**The best score is taking from Ridge regression. But for all model there is a high difference between Train and test score.**